# Approximate any molecule's ground state energy.

In this example notebook, we'll walk through calculating the energy of a molecule given an estimate for the ground state using Azure Quantum. We will do that using a simple algorithm where we prepare the quantum register in a state near the ground state of the molecule using the Jordan-Wigner encoding, and evaluate the energy of the molecule by applying the Hamiltonian operator to that state. To be able to run the algorithm on near-term hardware, we will estimate the energy per Hamiltonian term, and then calculate the total estimated energy by adding the values of these terms.

In [1]:
# First, import qsharp and qsharp.azure to be able to compile and submit the quantum program.
import qsharp
import qsharp.azure
qsharp.reload()
from Microsoft.Quantum.Chemistry.Hamiltonian import GetHamiltonianTerm, ExpandedCoefficients_, JordanWignerMeasurementOperators


/usr/local/lib/python3.9/site-packages/qutip/__init__.py:65: UserWarning: The new version of Cython, (>= 3.0.0) is not supported.
  warnings.warn(


Preparing Q# environment...
.

Reloading workspace.

### 1. Load molecule data

The pre-generated Broombridge file included in this sample contains the details on the molecule's Hamiltonian. A Hamiltonian is an operator that calculates the energy of a molecule by acting on a qubit register that represents the molecule's quantum state by Jordan-Wigner encoding. If we prepare the molecule in the ground state, then we know that applying the Hamiltonian will calculate the ground state energy.

Below are the sample files you could use:
1. ../data/broombridge/lithiumHydride.yaml
2. ../data/broombridge/caffeine.yaml
3. ../data/broombridge/hydrogen_0.2.yaml

You could also source your molecule file from NW chem library: [NWChem](https://learn.microsoft.com/en-us/azure/quantum/user-guide/libraries/chemistry/samples/end-to-end)

In [55]:
from qdk.chemistry.broombridge import load_and_encode

In [56]:
encoded_data = load_and_encode("../data/broombridge/caffeine.yaml")

This contains a tuple of the number of qubits, the fermionic Hamiltonian term coefficients and the energy offset.

The index of each list in the second tuple is will be referred to as the "term type".

In [57]:
num_qubits, fermion_terms, _, energy_offset = encoded_data

### 3. Running on Azure Quantum

Connect to Azure Quantum

In [5]:
rid = "" # Enter your workspace's resource ID here
location = "East US" # Enter your workspace's location here, e.g. "West US"

In [7]:
qsharp.azure.connect(
   resourceId=rid,
   location=location,
   credential="devicecode"
)

Connecting to Azure Quantum...

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FMA5ZSKBK to authenticate.


Connected to Azure Quantum workspace jkingdon-qw in location eastus.


[{'id': 'ionq.qpu', 'current_availability': {}, 'average_queue_time': 1157432},
 {'id': 'ionq.qpu.aria-1', 'current_availability': {}, 'average_queue_time': 105954},
 {'id': 'ionq.qpu.aria-2', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'ionq.simulator', 'current_availability': {}, 'average_queue_time': 139972},
 {'id': 'microsoft.estimator', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.qpu.h1-1', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.sim.h1-1sc', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.qpu.h1-2', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.sim.h1-2sc', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.sim.h1-1e', 'current_availability': {}, 'average_queue_time': 3},
 {'id': 'quantinuum.sim.h1-2e', 'current_availability': {}, 'average_queue_time': 88},
 {'id': 'rigetti.sim.qvm', 'current_availability': {}, 'average_qu

In [8]:
qsharp.azure.target("ionq.simulator")

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.simulator


{'id': 'ionq.simulator', 'current_availability': {}, 'average_queue_time': 139972}

### 4. Loop over all Hamiltonian terms

Each Hamiltonian term has a term type, measurement operator and coefficient. To simplify this sample, we will move the coefficients into a flat list. Each nonzero coefficient will render a separate job that we send to Azure Quantum. To calculate the energy for that term, we map result from the job onto a -1 to 1 axis and multiply the value by the corresponding coefficient.

We also calculate the measurement operators that we would need to measure the 

In [58]:
coeffs = []
measurementOps = []
for term_type, terms in enumerate(fermion_terms):
    print ("term_type", term_type)
    for (qubits, coeff) in terms:
        print("qubits", qubits, "coeff", coeff)
        coeffs += ExpandedCoefficients_(coeff=coeff, termType=term_type)
        measurementOps += JordanWignerMeasurementOperators(nQubits=num_qubits, indices=qubits, termType=term_type)

term_type 0
qubits [0] coeff [0.17120128499999998]
qubits [1] coeff [0.17120128499999998]
qubits [2] coeff [-0.222796536]
qubits [3] coeff [-0.222796536]
term_type 1
qubits [0, 1] coeff [0.1686232915]
qubits [0, 2] coeff [0.12054614575]
qubits [0, 3] coeff [0.16586802525]
qubits [1, 2] coeff [0.16586802525]
qubits [1, 3] coeff [0.12054614575]
qubits [2, 3] coeff [0.1743495025]
term_type 2
term_type 3
qubits [0, 1, 2, 3] coeff [0.0, -0.0453218795, 0.0, 0.0453218795]


You obtain the total number of coefficients corresponding to each combination of Hamiltonian term and Jordan-Wigner measurement operator using the function above. Now we start a job for each term that has a nonzero coefficient. We do this in batches and wait for the completion of that batch before moving onto the next batch as a qauntum simulator only has limited qubits to compute with.

In [60]:
jobs = []
batch_size = 5
current_batch_jobs = []

for n_op, coeff in enumerate(coeffs):
    if coeff != 0.0:
        current_batch_jobs.append(qsharp.azure.submit(GetHamiltonianTerm, measurementOps=measurementOps[n_op], nQubits=num_qubits, shots=1000, jobName=f"Hamiltonian term {n_op}"))
    if ((n_op + 1) % batch_size == 0) or (n_op == (len(coeffs) - 1)):
        # wait until all current jobs complete before submitting the next batch
        print("Waiting for jobs to complete...")
        while True:
            all_done = all([qsharp.azure.status(j.id).status == "Succeeded" for j in current_batch_jobs])
            if all_done:
                print("Completed job batch!")
                break
        # clear current batch jobs for next batch            
        current_batch_jobs = []

Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTerm to target ionq.simulator...
Job successfully submitted.
   Job name: Hamiltonian term 0
   Job ID: 7ecd711e-82e9-4c46-9e31-f0b86d1251d4
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTerm to target ionq.simulator...
Job successfully submitted.
   Job name: Hamiltonian term 1
   Job ID: d274a3f9-71ae-4547-8f6d-574ae060bae0
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTerm to target ionq.simulator...
Job successfully submitted.
   Job name: Hamiltonian term 2
   Job ID: 3f7ec75d-ece9-41c1-96e8-b6414b46fbd6
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTerm to target ionq.simulator...
Job successfully submitted.
   Job name: Hamiltonian term 3
   Job ID: 7e23d061-1da3-48ca-8a37-1e7496d37686
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTerm to target ionq.simulator...
Job successfully submitted.
   Job name: Hamiltonian term 4
   Job ID: 0318b45

In [61]:
results = [qsharp.azure.output(j.id) for j in jobs]
results

[]

### 5. Estimate energy

In [62]:
def calc_energy(coeffs, results):
    return sum([(2. * res.get("0", 0.0) - 1.) * coeff for coeff, res in zip(coeffs, results)]) + energy_offset

In [63]:
calc_energy(coeffs, results)

-0.09883444600000002

Now we are ready to run on hardware! Switch to the QPU.

In [ ]:
qsharp.azure.target("ionq.qpu")

In [ ]:
jobs_hw = [
    qsharp.azure.submit(GetHamiltonianTerm, nOp=n_op, measurementOps=measurementOps, shots=1000, jobName=f"Hamiltonian term {n_op}") 
    for n_op, coeff in enumerate(coeffs) if coeff != 0
]

Check if the jobs succeeded.

In [ ]:
jobs_status = [qsharp.azure.status(j) for j in jobs_hw]
[
    j.status + " | " + j.name + " | " + j.target + " | " + j.creation_time + " | " + j.end_execution_time 
    for j in jobs_status
]

Get results and estimate energy

In [ ]:
results = [qsharp.azure.output(j.id) for j in jobs_hw]
results

In [ ]:
calc_energy(coeffs, results)